In [1]:
print("Guys lets do this")

Guys lets do this


In [1]:
import pandas as pd

movies = pd.read_csv('movies.csv')
print(movies.head())


   movie_id                   movie_title movie_genres  \
0      3107              Backdraft (1991)         0, 7   
1      2114         Outsiders, The (1983)            7   
2       256                 Junior (1994)        4, 15   
3      1389               Jaws 3-D (1983)        0, 10   
4      3635  Spy Who Loved Me, The (1977)            0   

                                          poster_url  
0  https://image.tmdb.org/t/p/w500/bymJtYNIdiAiBR...  
1  https://image.tmdb.org/t/p/w500/pl8Tf36TAOb2i5...  
2  https://image.tmdb.org/t/p/w500/4tRF43CbEYHmV8...  
3  https://image.tmdb.org/t/p/w500/kqDXj53F9paqVG...  
4  https://image.tmdb.org/t/p/w500/3ZxHKFxMYvAko6...  
